## Sweep bot

i'm fucking crashing out
why is nothing fucking working

In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [13]:

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    ConstantVectorSource,
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    TrajectorySource,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
    PublishPositionTrajectory,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [22]:
from scripts.final_hail_mary import plan_sweep
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario(use_position=True, use_weld=True)

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)
        # AddDefaultVisualization(parser.builder(), meshcat)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()

    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    plant_context = plant.CreateDefaultContext()
    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa'))
    broom_frame = plant.GetFrameByName("handle_link")
    X_WStart = broom_frame.CalcPoseInWorld(plant_context)
    X_WGoal = RigidTransform(RotationMatrix(), np.array([0.2, 0, 0]))
    AddMeshcatTriad(meshcat, 'goal', X_PT=X_WGoal)

    trajectory = plan_sweep(broom_frame, q0, X_WStart, X_WGoal)
    traj_src = builder.AddSystem(
        TrajectorySource(trajectory)
    )

    wsg_src = builder.AddSystem(ConstantVectorSource([0.5]))

    # builder.Connect(
    #     demux.get_output_port(0),
    #     control_desired_state_input_port
    # )

    builder.Connect(
        traj_src.get_output_port(),
        station.GetInputPort('iiwa.position')
    )

    builder.ExportOutput(traj_src.get_output_port(), 'pos')

    builder.Connect(
        wsg_src.get_output_port(),
        station.GetInputPort("wsg.position")
    )


    diagram = builder.Build()

    return diagram, plant, station 

# diagram = builder.Build()

In [23]:
diagram, plant, station  = make_diagram()

7
(7, 10)
Solver failed
['path position constraint at s=0[0]: 0.014904 <= -0.102350 <= 0.014904', 'path position constraint at s=0[1]: 0.719573 <= 0.578747 <= 0.719573', 'path position constraint at s=0[2]: 0.223269 <= 0.305201 <= 0.223269', 'path position constraint at s=1[0]: 0.200000 <= -0.145107 <= 0.200000', 'path position constraint at s=1[1]: 0.000000 <= 1.564786 <= 0.000000', 'path position constraint at s=1[2]: 0.000000 <= 1.120096 <= 0.000000', 'path position constraint at s=0[2]: -inf <= 0.305201 <= 0.050000', 'path position constraint at s=0.05263157894736842[2]: -inf <= 0.266939 <= 0.050000', 'path position constraint at s=0.10526315789473684[2]: -inf <= 0.201947 <= 0.050000', 'path position constraint at s=0.15789473684210525[2]: -inf <= 0.180865 <= 0.050000', 'path position constraint at s=0.21052631578947367[2]: -inf <= 0.203836 <= 0.050000', 'path position constraint at s=0.2631578947368421[2]: -inf <= 0.202704 <= 0.050000', 'path position constraint at s=0.31578947368

In [ ]:

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)
controller.add_diagram(diagram, context, meshcat)
iiwa = plant.GetModelInstanceByName('iiwa')
# broom = plant.GetModelInstanceByName('broom')
# iiwa_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("base", iiwa))
# broom_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("handle_link"))
# AddMeshcatTriad(meshcat, 'iiwa', X_PT=iiwa_pose)
# AddMeshcatTriad(meshcat, 'broom', X_PT=broom_pose)

simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
end_time = 3.4
time = 0

simulator.AdvanceTo(2)

meshcat.StopRecording()
meshcat.PublishRecording()
# controller.sub_meshcat.StopRecording()
# controller.sub_meshcat.PublishRecording()

[[-0.13621077 -0.13463403 -0.1330578  ... -0.04846403 -0.04647926
  -0.04449448]
 [ 0.8001246   0.8001006   0.80007666 ...  0.28342864  0.28342864
   0.28342864]
 [ 0.26109767  0.26120996  0.26132256 ...  0.0499999   0.0499999
   0.0499999 ]]
empty trajectories at time 0.0
called at time 0.0
start!
7
(7, 10)
Solver failed
['path position constraint at s=0[0]: 0.014904 <= 0.014059 <= 0.014904', 'path position constraint at s=0[1]: 0.719573 <= 0.720164 <= 0.719573', 'path position constraint at s=0[2]: 0.223269 <= 0.223511 <= 0.223269', 'path position constraint at s=1[0]: 2.999999 <= 2.969556 <= 3.000000', 'path position constraint at s=0[0]: -inf <= 2.480758 <= 1.000000', 'path position constraint at s=0.7083333333333333[0]: -inf <= 469.656904 <= 1.000000', 'path position constraint at s=0.75[0]: -inf <= 13.975641 <= 1.000000', 'path position constraint at s=0.9166666666666666[0]: -inf <= 1.382152 <= 1.000000']
attempts needed: 1
[1.0, 0.0, 0.0, 0.0]
will end at: 3.197578433494916
